In [1]:
## RF Study for Iridium-Next constellation-Tolosat Payload
# TOLOSAT
# Link budget
#Adapted from a 12/2020 MATLAB code by Nicola imperaatore, 2020-2021 Iridium subsystem leader
# Code for the calculation of a link budget TOLOSAT (/ground) - Iridium

#mag/db conversions
import control
# general library import
import numpy as np

# TLE tools import
from tletools import TLE

# poly astro library import
from astropy import units as u
import poliastro as pa

# plotting utilities
import plotly.io as pio
from poliastro.plotting import OrbitPlotter3D

# to plot at current time
from astropy.time import Time


In [2]:
## INPUTS
uplink = False
#Average Power during a transmit slot (max) [DR1 p25]
P_PL = 1.5 #[W] #It is 1.6W for the 9602 modem
#Average power emitted by an iridium satellite

########### UNKNOWN ###########
### NEED TO ASK V. MARTELET ###
###############################
P_SAT = 1

#Gain max of the payload/ground/our antenna
#MPA-D254-1621 realized gain at 1621 Hz [DR2]
GPL_max = 2.5 #[dBi]

#No clue on how to estimate system noise temperature. We assume this number
#from Iridium budget sheet
#####TO BE REEVALUATED#####
GoverT_PL = -31    #[dB] [DR3]

#Gain max of Iridium satellite
# Phased antenna array
#####TO BE VERIFIED#####
GSAT_max = 20.9 #[dB]

# We assume this number
#from Iridium budget sheet
GoverT_SAT = -11.2    #[dB] [DR4]
###########ON D its written -13.6!!!!! --->but need to undertsand what is Statistical value @90% earth for " G/T *Xpol_loss"

##################"to be deleted#####################
#Elevation angle (case ground to satellite)
#E = 23.53*np.pi/180     #[rad]
#Difference in true anomalies between TOLOSAT and Iridum next satellite
#(case satellite to satellite)
#Dth = 10*np.pi/180   #[rad]
#Earth radius
RE = 6378e3   #[m]
#Altitude of Iridium satellite orbit
#H = 780e3 #[m]


#Bandwidth 
B = 10.5e6  #[Hz] [DR1]
#Symbol rate 
SR = 25e3  #[bps] [DR 3, 4]
#Modulation size
M = 4  #[DR 5]
#Data rate 
DR=3,5  #[kb/s/burst] [DR 3, 4]
#Link frequency
f0 = 1621*1e6   #[Hz] [DR1 p25]

#Speed of light
c = 2.998e8   #[m/s]
#Boltzmann constant
kB = -228.8 #[dBJ/K]

#Maximum cable losses admitted between modem and antenna
Lc_PL = 2    #[dB] # [DR1 p25]
#NB:Solutions with a loss higher than 3dB will not meet the
#requirements of Iridium Solution certification.

#Losses in Iridium satellite (no clue how to model this)
Lc_SAT = 0    #[dB]

####NEED TO BE VERIFIED WITH THE ANALYSIS
#Axial ratio of our MPA-D254-1621 antenna 
AR_PL = 4 #[dB] [DR2]

#Axial ratio of the Iridium satellite antennas
#Iridium next satellite axial ratio (Iridium constellation document)
AR_SAT = 3.5  #[dB] [DR6 p15]

#Atmospheric losses
#Negligible @f0
##DON'T UNDERSTAND WHAT IT IS YET
Latm = 0.33   #[dB] [DR3]

#Diverse losses, margin
#Term on modem documentation
Lm_UL = 0 #[dB]
Lm_DL =0  #[dB]

Uplink or downlink


In [3]:
if(uplink):
    Pin = P_PL
    Gt_max = GPL_max
    Gr_max = 0
    ARt = control.db2mag(AR_PL)
    ARr = control.db2mag(AR_SAT)
    Ltx = Lc_PL
    Lrx = Lc_SAT
    Lm = Lm_UL
    GoverT = GoverT_SAT
else :
    Pin = P_SAT
    Gt_max = GSAT_max
    Gr_max = GPL_max
    ARt = control.db2mag(AR_SAT)
    ARr = control.db2mag(AR_PL)
    Ltx = Lc_SAT
    Lrx = Lc_PL
    Lm = Lm_DL
    GoverT = GoverT_PL

Code by @benoit-maillet to calculate the minimum distance from Tolosat to an iridium-next satellite.

In [56]:
# this converts list of TLE to list of poliastro.orbit

iridium_TLE = TLE.load('iridium-next.txt') # load TLE

iridium_orb = [] # initialization

for i in range(len(iridium_TLE)): # convertion
    iridium_orb.append(iridium_TLE[i].to_orbit())


# function to get the distance between two object
def dist_between(a, b):
    return np.sqrt( (a.r[0]-b.r[0])**2 + (a.r[1]-b.r[1])**2 + (a.r[2]-b.r[2])**2)

now = Time.now() # get the actual time

for i in range(len(iridium_orb)):
    iridium_orb[i] = iridium_orb[i].propagate(now)

tolosat_orb = pa.twobody.orbit.Orbit.from_classical(
    pa.bodies.Earth,    # main attractor
    6878 * u.km,        # Semi-major axis
    0.002 * u.one,      # Eccentricity
    97.4 * u.deg,       # Inclination
    0.187 * u.rad,      # RAAN (Right ascension of the ascending node)
    np.pi/2 * u.rad,    # Argument of the pericenter
    0 * u.deg,          # true anomaly
    now                 # epoch (time)
)
distFromConst = []
distFromConst_name = []

for i in range(len(iridium_orb)):
    distFromConst.append(dist_between(tolosat_orb,iridium_orb[i]))
    distFromConst_name.append(iridium_TLE[i].name)

index_of_min = distFromConst.index(min(distFromConst))
##Question: is the closer ir sat the one we have connexion with?
## I have an idea: lets imagine Tolosat has its velocit vector against Iridium plan it is seeing. It is close to
# one plan but its velocity is higher so he could not establish a com with this plan but if it see the otheer plan it s slower compared to it so com available
R=min(distFromConst) #
print(R.value)

259.77584121440765


Preliminary calculations

In [58]:
#Input power conversion
P_PL = control.mag2db(P_PL)
#Wavelength
lambd = 1621*1e6 / f0 #[m]

#Atmospheric loss
Lfs = (4*np.pi*R.value)**2 / lambd**2
Lfs = control.mag2db(Lfs)
print(Lfs)

#Polarisation loss
#Beta is the angle between transmitter and receiver ellipses
beta = 90*np.pi/180
Lpol = 1/2 + 2*ARt*ARr / ( (1+ ARt**2)*(1+ARr**2))+ ((1-ARt**2)*(1-ARr**2) / (2* (1+ARr**2)*(1+ARr**2)))*np.cos(2*beta)
Lpol = -control.mag2db(Lpol)
print(Lpol)


140.55234495459376
1.5034991823586803


In [59]:


## Link budget
L =  Lfs + Lpol + Ltx + Lrx + Latm + Lm # Total loss

Prec = Pin + Gt_max + Gr_max - L # Link budget equation in dB

control.db2mag(Prec) #W
control.mag2db(control.db2mag(Prec)*1e3) #dBm

#carrier-to-noise ratio ie
#signal-to-noise ratio (SNR) of the received signal
# after the receiver filter but before detection:
####IS THERE AN ERROR THERE WITH kB####
CN0 = Pin+ Gt_max + GoverT - L - kB 

#energy per symbol to noise power spectral density 
#Signal to noise ratio
###Isnt'it EsN0=B/SR
EsN0 = CN0 - control.mag2db(SR)
#Eb/N0 (energy per bit to noise power spectral density ratio)
#Normalized SNR
EbN0 = EsN0 - control.mag2db(np.log2(M))